### 语义分割之 U-Net 详解

#### 网络图

#### U-Net 网络代码

In [1]:
import torch 
import torch.nn as nn

from torch import autograd

In [ ]:
# 把常用的两个卷积封装在一起

class DoubleConve(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(DoubleConve,self).__init__()
        
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            
        )
        
        